## 0. Imports and Configs


In [ ]:
import pandas as pd
import streamlit as st
import os


# CONFIG: qdrant
api_key = st.secrets["QDRANT_API_KEY"]
url = st.secrets["QDRANT_URL"]  # for cloud
qdrant_collection_name = "ASK_vectorstore"
# for local instance /private/tmp/local_qdrant
qdrant_path = "/Users/drew_wilkins/Drews_Files/Drew/Python/VSCode/ASK/data/qdrant"

## 1. Get the collection and records

In [4]:
# Load an instance of the client. WITHOUT LANGCHAIN
# 22.5 sec for cloud
# Running this locally places a lock file in the qdrant directory
from qdrant_client import QdrantClient

client = QdrantClient(url,
                      prefer_grpc=True,
                      api_key=api_key,
                      )

'''usage'''
content = (client.get_collections())

In [ ]:
def create_all_record_dict():
    all_records = client.scroll(
        collection_name=qdrant_collection_name,
        limit=100000,
    )
    records = all_records[0]

    # Initialize an empty dictionary to store the results
    records_dict = {}

    # Iterate through each record in the records list
    for record in records:
        # Extract the id and metadata from the record
        record_id = record.id
        metadata = record.payload['metadata']

        # Exclude the 'page_content' from the metadata
        if 'page_content' in record.payload:
            del record.payload['page_content']

        # Add the record to the records_dict
        records_dict[record_id] = metadata

    return all_records_dict


all_records_dict = create_all_record_dict()

## 2. Check for Duplicates  
#### NOTE: In a few circumstances, you may have multiple chunks for one page, for example if the content is dense. This would throw a duplicate error below. In these cases, check the actual page content to detemine whether there is actually a duplicate.

In [ ]:
def find_duplicate_pdfs_in_records(all_records_dict):
    seen = {}  # Dictionary to keep track of source-page combinations
    duplicate_record_count = 0  # Counter to track the number of duplicate records
    duplicate_pdf_sources = set()  # Set to track unique sources of duplicate PDFs
    duplicate_details = []  # List to store details of each duplicate pair

    for record_id, record in all_records_dict.items():
        source = record['source']
        page = record['page']

        # Create a tuple to represent the combination of source and page
        key = (source, page)

        if key in seen:
            # Increment the duplicate record count
            duplicate_record_count += 1

            # Add the source to the set of duplicate PDFs
            duplicate_pdf_sources.add(source)

            # Store the original and duplicate records for detailed output later
            original_record = seen[key]
            duplicate_details.append({
                "original": {"id": original_record["id"], "source": original_record["source"], "page": original_record["page"]},
                "duplicate": {"id": record_id, "source": source, "page": page}
            })
        else:
            # Store this unique (source, page) combination along with the record ID
            seen[key] = {"id": record_id, "source": source, "page": page}

    # Calculate the number of unique PDFs that have duplicates
    duplicate_pdf_count = len(duplicate_pdf_sources)

    # Output the requested information
    print(f"DUPLICATE PDFs FOUND: {duplicate_pdf_count}")
    print("")
    for pdf_name in duplicate_pdf_sources:
        print(pdf_name, "\n")
    print("\nDUPLICATE RECORDS:", duplicate_record_count, "\n")

    # Print each duplicate pair as specified
    for detail in duplicate_details:
        original = detail["original"]
        duplicate = detail["duplicate"]
        print("Original:")
        print(
            f"id: {original['id']}, source: {original['source']}, page: {original['page']}")
        print("Duplicate:")
        print(
            f"id: {duplicate['id']}, source: {duplicate['source']}, page: {duplicate['page']}")
        print("")  # Blank line between duplicate pairs for readability


# Example usage
duplicates = find_duplicate_pdfs_in_records(all_records_dict)

## 3. Get unique records (list of PDFs)

In [18]:
def create_unique_sources_dict(all_records_dict):

    # Use a set to keep track of unique sources
    seen_sources = set()

    # Dictionary to store records with unique sources
    unique_dict = {}

    # Iterate through each record in the all_records_dict
    for record_id, metadata in all_records_dict.items():
        # If the record's source is not in the set, add it to the set and the unique_dict
        source = metadata['source']
        if source not in seen_sources:
            seen_sources.add(source)
            unique_dict[record_id] = metadata

    return unique_dict


'''usage'''
unique_sources_dict = create_unique_sources_dict(all_records_dict)
print("Number of PDFs:", len(unique_sources_dict))

Number of PDFs: 251


## 4. Format the List

In [19]:
def format_unique_source_dict(unique_sources_dict):
    '''Format values and store as a dataframe'''

    formatted_list = []

    for record_id, metadata in unique_sources_dict.items():
        # Extract the base filename without the extension
        source = metadata['source']
        base_filename = os.path.splitext(os.path.basename(source))[0]
        metadata['source_short'] = base_filename

        # Append the metadata to the list
        formatted_list.append(metadata)

        # exclude all metadata from list except these (source will be removed once we have all the metadata
        # metadata = {key: metadata[key] for key in ['short_source', 'source'] if key in metadata}

    # Convert the list to a DataFrame
    dataframe = pd.DataFrame(formatted_list)

    # Remove the 'page' column
    dataframe = dataframe.drop(columns=['page'])
    return dataframe


'''usage'''
unique_sources_df = format_unique_source_dict(unique_sources_dict)

## 5. Save the List to Excel

In [ ]:
from datetime import datetime, timezone


def write_library_xlsx(unique_sources_df):
    """write dataframe to an Excel file."""

    # Get the current date and time in Zulu (UTC) time
    now_utc = datetime.now(timezone.utc)
    timestamp = now_utc.strftime('%d%b%Y-%H%M')

    # Specify the relative path to save the Excel file with the timestamp appended
    file_path = f'library_docs_list_{timestamp}.xlsx'

    # Save DataFrame to Excel
    unique_sources_df.to_excel(file_path, index=False)


'''usage'''
write_library_xlsx(unique_sources_df)